In [1]:
import numpy as np
import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kieran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kieran\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
amazondf = pd.read_csv("amazon.csv", error_bad_lines=False)
amazondf

c:\Users\kieran\anaconda3\envs\python3.7\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,₹379,₹919,59%,4,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1461,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,"₹2,280","₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but the me

In [3]:
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace("₹", "")
amazondf["discounted_price"] = amazondf["discounted_price"].str.replace(",", "")
amazondf["discounted_price"] = amazondf["discounted_price"].astype("float32")

amazondf["actual_price"] = amazondf["actual_price"].str.replace("₹", "")
amazondf["actual_price"] = amazondf["actual_price"].str.replace(",", "")
amazondf["actual_price"] = amazondf["actual_price"].astype("float32")

amazondf.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [4]:
amazondf.drop_duplicates(inplace = True)

In [5]:
pricedf = amazondf[["product_id", "product_name", "category", "about_product", "discounted_price", "actual_price"]].copy()
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,High Compatibility : Compatible With iPhone 12...,399.0,1099.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"Compatible with all Type C enabled devices, be...",199.0,349.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0


In [6]:
# pricedf['category'] = pricedf['category'].apply(lambda x: " ".join(x.split('|')))

# obtain the last category from the dataset
pricedf["category"] = pricedf['category'].apply(lambda x: x[(x.rfind('|'))+1:])
pricedf["category"].value_counts()
# pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables,High Compatibility : Compatible With iPhone 12...,399.0,1099.0
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables,"Compatible with all Type C enabled devices, be...",199.0,349.0
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0


In [7]:
# create 7 descriptors and compare their results

pricedf["description1"] = pricedf["product_name"] 
pricedf["description2"] = pricedf["about_product"]
pricedf["description3"] = pricedf["category"]
pricedf["description4"] = pricedf["product_name"] + " " + pricedf["about_product"]
pricedf["description5"] = pricedf["product_name"] + " " + pricedf["category"]
pricedf["description6"] = pricedf["about_product"] + " " + pricedf["category"]
pricedf["description7"] = pricedf["product_name"] + " " + pricedf["about_product"] + " " + pricedf["category"]
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description,name_description
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,USBCables Wayona Nylon Braided USB to Lightnin...,Wayona Nylon Braided USB to Lightning Fast Cha...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables,"Compatible with all Type C enabled devices, be...",199.0,349.0,USBCables Ambrane Unbreakable 60W / 3A Fast Ch...,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,USBCables Sounce Fast Phone Charging Cable & D...,Sounce Fast Phone Charging Cable & Data Sync U...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,USBCables boAt Deuce USB 300 2 in 1 Type-C & M...,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,USBCables Portronics Konnect L 1.2M Fast Charg...,Portronics Konnect L 1.2M Fast Charging 3A 8 P...


In [8]:
def remove_stopwords(sentence):
    stop = set(stopwords.words('english'))
    processed = " ".join([word for word in sentence.split() if word not in stop])
    return processed

def remove_urls(sentence):
    processed = re.sub('http[s]?://(?:[a-zA-Z]|[0–9]|[$-_@.&+#]|[!*\(\),]|'\
                   '(?:%[0–9a-fA-F][0–9a-fA-F]))+','', sentence)
    return processed

def remove_punctuation(sentence):
    punc = string.punctuation
    sentence = sentence.translate(str.maketrans('', '', punc))
    return sentence


In [9]:
lst = ["description1", "description2", "description3", "description4", "description5", "description6", "description7"]

for description in lst:
    pricedf[description] = pricedf[description].apply(lambda x: remove_urls(x))
    pricedf[description] = pricedf[description].apply(lambda x: x.lower())
    pricedf[description] = pricedf[description].apply(lambda x: remove_stopwords(x))
    pricedf[description] = pricedf[description].apply(lambda x: remove_punctuation(x))

    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_urls(x))
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: x.lower())
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_stopwords(x))
    # pricedf["name_description"] = pricedf["name_description"].apply(lambda x: remove_punctuation(x))

# pricedf.head()

lemmatizer = WordNetLemmatizer()
def lemmatization(sentence):
    sentence = ' '.join([lemmatizer.lemmatize(word) for word in sentence.split()])
    return sentence

for description in lst:
    pricedf[description] = pricedf[description].apply(lambda x: lemmatization(x))
# pricedf["name_description"] = pricedf["name_description"].apply(lambda x: lemmatization(x))
pricedf.head()

,product_id,product_name,category,about_product,discounted_price,actual_price,description,name_description
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,USBCables,High Compatibility : Compatible With iPhone 12...,399.0,1099.0,usbcables wayona nylon braided usb lightning f...,wayona nylon braided usb lightning fast chargi...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,USBCables,"Compatible with all Type C enabled devices, be...",199.0,349.0,usbcables ambrane unbreakable 60w 3a fast cha...,ambrane unbreakable 60w 3a fast charging 15m ...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,USBCables,【 Fast Charger& Data Sync】-With built-in safet...,199.0,1899.0,usbcables sounce fast phone charging cable da...,sounce fast phone charging cable data sync us...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,USBCables,The boAt Deuce USB 300 2 in 1 cable is compati...,329.0,699.0,usbcables boat deuce usb 300 2 1 typec micro ...,boat deuce usb 300 2 1 typec micro usb stress...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,USBCables,[CHARGE & SYNC FUNCTION]- This cable comes wit...,154.0,399.0,usbcables portronics konnect l 12m fast chargi...,portronics konnect l 12m fast charging 3a 8 pi...


In [11]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

for description in lst:
    X_train, X_test, y_train, y_test = train_test_split(pricedf[description], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

    print(description)
    tv = TfidfVectorizer()
    # train_tfidf = tv.fit_transform(ratingdf["combined_review"]).toarray()
    X_train_tfidf = tv.fit_transform(X_train) # fit train
    X_test_tfidf = tv.transform(X_test) # transform test
    print(X_train_tfidf.shape, X_test_tfidf.shape)


    print("----- linear regression -----")
    linear = LinearRegression()
    linear = make_pipeline(StandardScaler(with_mean=False), linear)
    linear.fit(X_train_tfidf, y_train)
    y_pred = linear.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))
    print(linear.score(X_test_tfidf, y_test))
    print(y_pred[:10])
    print(y_test[:10])


    print("\n ----- support vector regression -----")
    svr = SVR(kernel='linear')
    svr = make_pipeline(StandardScaler(with_mean=False), svr)
    svr.fit(X_train_tfidf, y_train)
    print(svr.score(X_test_tfidf, y_test))

    y_pred = svr.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))

    print(y_pred[:10])
    print(y_test[:10])


    print("\n ----- ridge regression -----")
    ridge = Ridge(alpha = 0.5)
    ridge.fit(X_train_tfidf, y_train)
    print(ridge.score(X_test_tfidf, y_test))

    y_pred = ridge.predict(X_test_tfidf)
    print(r2_score(y_pred, y_test))

    print(y_pred[:10])
    print(y_test[:10])



Linear Regression

In [14]:
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler

linear = LinearRegression()
linear = make_pipeline(StandardScaler(with_mean=False), linear)
linear.fit(X_train_tfidf, y_train)
y_pred = linear.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

print(linear.score(X_test_tfidf, y_test))

print(y_pred[:10])
print(y_test[:10])

0.7501416295206371
0.7656820404570392
[20404.98872328 -2565.97629095 27807.15167594  3574.68016531
  9870.79012737   747.1156639   3270.86576992  5100.66708538
 -1982.98555193  3245.59097082]
434     20999.0
194       599.0
82      19125.0
294       899.0
1313     4495.0
1268      980.0
317       399.0
613      1995.0
320       599.0
117       999.0
Name: actual_price, dtype: float32


Support Vector Regression

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
# sc = StandardScaler()
# X_train_sc = sc.fit_transform(X_train_tfidf)
# y_train_sc = sc.fit_transform(y_train)
# X_test_sc = sc.transform(X_test_tfidf)
# y_test_sc = sc.transform(y_test)

svr = SVR(kernel='linear')
# svr = make_pipeline(StandardScaler(with_mean=False), svr)
svr.fit(X_train_tfidf, y_train)
print(svr.score(X_test_tfidf, y_test))

y_pred = svr.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

print(y_pred[:10])
print(y_test[:10])

-0.12830281069963578
-783452.6362416778
[1664.20639662 1624.94747075 1679.13484018 1629.28746658 1656.72602025
 1652.50169965 1645.44772271 1649.23417201 1643.140167   1637.51839386]
434     20999.0
194       599.0
82      19125.0
294       899.0
1313     4495.0
1268      980.0
317       399.0
613      1995.0
320       599.0
117       999.0
Name: actual_price, dtype: float32


In [16]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train_tfidf, y_train)
print(ridge.score(X_test_tfidf, y_test))

y_pred = ridge.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

print(y_pred[:10])
print(y_test[:10])

0.9115981185524789
0.9093979617559842
[19741.36842823  -301.83762457 36347.36212486  5913.30000102
  6756.18773027  2845.36132587 -4333.47395499  2390.61836437
  -157.98781228  1896.60429494]
434     20999.0
194       599.0
82      19125.0
294       899.0
1313     4495.0
1268      980.0
317       399.0
613      1995.0
320       599.0
117       999.0
Name: actual_price, dtype: float32


In [17]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge = make_pipeline(StandardScaler(with_mean=False), ridge)
ridge.fit(X_train_tfidf, y_train)
print(ridge.score(X_test_tfidf, y_test))

y_pred = ridge.predict(X_test_tfidf)
print(r2_score(y_pred, y_test))

y_pred = [0 if y< 0 else y for y in y_pred]
print(r2_score(y_pred, y_test))

print(y_pred[:10])
print(y_test[:10])

0.9058353740823721
0.8945142241974868
0.8951131347645915
[20357.443667216918, 0, 27349.04375186894, 5483.798655128651, 5092.019149949978, 695.2861451148779, 1479.1800710140833, 1613.0171875025017, 104.0683107009254, 2990.561753642749]
434     20999.0
194       599.0
82      19125.0
294       899.0
1313     4495.0
1268      980.0
317       399.0
613      1995.0
320       599.0
117       999.0
Name: actual_price, dtype: float32


# BERT

X_train, X_test, y_train, y_test = train_test_split(pricedf["description"], pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

In [18]:
# pricedf.head()

pricedf['product_name'].head(10)

0    Wayona Nylon Braided USB to Lightning Fast Cha...
1    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
2    Sounce Fast Phone Charging Cable & Data Sync U...
3    boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...
4    Portronics Konnect L 1.2M Fast Charging 3A 8 P...
5    pTron Solero TB301 3A Type-C Data and Fast Cha...
6    boAt Micro USB 55 Tangle-free, Sturdy Micro US...
7               MI Usb Type-C Cable Smartphone (Black)
8    TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...
9    Ambrane Unbreakable 60W / 3A Fast Charging 1.5...
Name: product_name, dtype: object

In [19]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('all-mpnet-base-v2')

In [20]:
# flatten text list
flatten_list = lambda y:[x for a in y for x in flatten_list(a)] if type(y) is list else [y]
description_list = flatten_list(pricedf['description'].values.tolist())
price_list = flatten_list(pricedf['actual_price'].values.tolist())

print(description_list[:10])


['usbcables wayona nylon braided usb lightning fast charging data sync cable compatible iphone 13 1211 x 8 7 6 5 ipad air pro mini 3 ft pack 1 grey', 'usbcables ambrane unbreakable 60w 3a fast charging 15m braided type c cable smartphones tablet laptop type c device pd technology 480mbps data sync quick charge 30 rct15a black', 'usbcables sounce fast phone charging cable data sync usb cable compatible iphone 13 1211 x 8 7 6 5 ipad air pro mini io device', 'usbcables boat deuce usb 300 2 1 typec micro usb stress resistant tanglefree sturdy cable 3a fast charging 480mbps data transmission 10000 bend lifespan extended 15m lengthmartian red', 'usbcables portronics konnect l 12m fast charging 3a 8 pin usb cable charge sync function iphone ipad grey', 'usbcables ptron solero tb301 3a typec data fast charging cable made india 480mbps data sync strong durable 15meter nylon braided usb cable typec device charging adapter black', 'usbcables boat micro usb 55 tanglefree sturdy micro usb cable 3a 

In [21]:
# database sentences
description_embeddings = model.encode(description_list)
print(description_embeddings.shape)

# faq sentences
# faq_embeddings = model.encode(faq_list)
# print(faq_embeddings.shape)

(1465, 768)


In [22]:
# X_train, X_test, y_train, y_test = train_test_split(p, pricedf["actual_price"],  test_size=0.2, random_state=11, shuffle=True, stratify=None)

X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(description_embeddings, price_list,  test_size=0.2, random_state=11, shuffle=True, stratify=None)

print(X_train_bert.shape, X_test_bert.shape)
print(len(y_train_bert), len(y_test_bert))

(1172, 768) (293, 768)
1172 293


In [23]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train_bert, y_train_bert)
y_pred = linear.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(linear.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])


0.5240056962211592
0.18537886482611943
[22205.22   10257.182  42513.113   -576.8882  4998.149  20626.166
  9302.553  11359.994  -5076.107  -1399.6069]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
sc = StandardScaler()
# X_train_sc = sc.fit_transform(X_train_tfidf)
# y_train_sc = sc.fit_transform(y_train)
# X_test_sc = sc.transform(X_test_tfidf)
# y_test_sc = sc.transform(y_test)

svr = SVR(kernel='linear')
svr.fit(X_train_bert, y_train_bert)
y_pred = svr.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(svr.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])

-74400.22410318966
-0.12697418175933062
[1692.77397533 1558.50677605 1689.08721398 1555.95639853 1643.60942892
 1661.38845548 1639.40200502 1612.53044722 1618.16922591 1572.69374056]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]


In [25]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train_bert, y_train_bert)
y_pred = ridge.predict(X_test_bert)
print(r2_score(y_pred, y_test_bert))

print(ridge.score(X_test_bert, y_test_bert))

print(y_pred[:10])
print(y_test_bert[:10])

0.7596785521969819
0.809147098299243
[24838.428    2765.8347  31522.521      47.85791  9722.013    5821.8003
 -3952.125     852.69727  -882.8125    544.01855]
[20999.0, 599.0, 19125.0, 899.0, 4495.0, 980.0, 399.0, 1995.0, 599.0, 999.0]
